## Limpieza de datos pretest CAP

In [1]:
import pickle
import re
import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime

import sys
# temp = sys.stdout                 # store original stdout object for later
# sys.stdout = open('log_directivos.txt', 'w') # redirect all prints to this log file

## Estudiantes

ruta =Path().cwd()

with open(ruta/'columnas_pre_CAP.pickle', 'rb') as handle:
    dict_cap = pickle.load(handle)

sheet_id = "1cgtlX-mHsUYBux4oPUL3TTaPhdEWXF2V3qG2_t9_E7Q"
sheet_name = "Respuestas de formulario 1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
url = url.replace(" ", "%20")



In [2]:
dict_cap

{'Marca temporal': 'Marca temporal',
 'Digita tu número de cédula': 'CC',
 '¿Cómo considera usted que las comunidades de aprendizaje pueden contribuir a su labor docente? ': '1. Cómo pueden las comunidades de aprendizaje contribuir a su labor',
 'Agregue comentarios que considere importantes para complementar su respuesta anterior.': 'Comentarios contribución a su labor',
 '¿Ha estado involucrado previamente en comunidades de aprendizaje?': '2.1 Involucrado previamente en comunidades de aprendizaje',
 '¿En cuál comunidad ha estado involucrado previamente?': '2.2 Comunidad de aprendizaje previa',
 'Por favor evalúe los siguientes enunciados de acuerdo con su experiencia:  [Sé cómo resolver los problemas técnicos cuando fallan las TIC]': '3.01 Sé cómo resolver los problemas técnicos cuando fallan las TIC',
 'Por favor evalúe los siguientes enunciados de acuerdo con su experiencia:  [Puedo aprender sobre nuevas tecnologías fácilmente]': '3.02 Puedo aprender sobre nuevas tecnologías fácilm

dict_cap['¿Ha estado involucrado previamente en comunidades de aprendizaje?'] = '2.1 Involucrado previamente en comunidades de aprendizaje'
dict_cap['¿En cuál comunidad ha estado involucrado previamente?']='2.2 Comunidad de aprendizaje previa'
with open(ruta/'columnas_pre_CAP.pickle', 'wb') as handle:
    pickle.dump(dict_cap,handle)


In [3]:
def divide_question_YES_NO(num_col, str_col, opciones, df):
    for i, op in enumerate(opciones):
        #print(num_col, i)
        subpreg = f'{num_col}.{i+1} {op}'
        mask = df[str_col].str.contains(op, na=False)
        df[subpreg] = mask.copy()
        df.loc[:, subpreg] = df.loc[:, subpreg].replace({True: 'SI', False: 'NO'}).copy()


In [4]:
cap=pd.read_csv(url)
cap = cap.rename(columns=dict_cap)
cap = cap.dropna(subset='CC')


In [5]:
cap.columns

Index(['Marca temporal', 'CC',
       '1. Cómo pueden las comunidades de aprendizaje contribuir a su labor',
       'Comentarios contribución a su labor',
       '2.1 Involucrado previamente en comunidades de aprendizaje',
       '2.2 Comunidad de aprendizaje previa',
       '3.01 Sé cómo resolver los problemas técnicos cuando fallan las TIC',
       '3.02 Puedo aprender sobre nuevas tecnologías fácilmente',
       '3.03 Sé cómo usar las TIC con los estudiantes en clase',
       '4. Comentarios primera sección', '3.04 Usar el correo electrónico',
       '3.05 Crear y usar de modelos y simulaciones',
       '3.06 Automatizar tareas', '3.07 Usar Word', '3.08 Procesar Datos',
       '3.09 Resolver problemas a través de herramientas computacionales (como simulaciones)',
       '3.10 Resolver problemas a través de herramientas computacionales (como lenguajes de programación)',
       '5.01 Me apoyo en mis colegas para resolver problemas sobre cómo trabajar algún tema',
       '5.02 Puedo ha

In [6]:
cap['1. Cómo pueden las comunidades de aprendizaje contribuir a su labor'] = cap['1. Cómo pueden las comunidades de aprendizaje contribuir a su labor'].str.replace('STEM','stem')
cap['1. Cómo pueden las comunidades de aprendizaje contribuir a su labor'] = cap['1. Cómo pueden las comunidades de aprendizaje contribuir a su labor'].str.replace('Comunidades de Aprendizaje','comunidades de aprendizaje')

In [7]:
respuestas = list(set(cap['1. Cómo pueden las comunidades de aprendizaje contribuir a su labor']))


In [8]:
op_1 = []
for r in respuestas:
    r_2 = re.findall('[A-Z][^A-Z]*', r)
    for x in r_2:
        x_2 = x.strip(" ,")
        if x_2 not in op_1:
            op_1.append(x_2)
len(op_1)




14

In [9]:
opciones =op_1
col = '1. Cómo pueden las comunidades de aprendizaje contribuir a su labor'

divide_question_YES_NO(1, col, opciones, cap)

In [10]:
cap

,Marca temporal,CC,1. Cómo pueden las comunidades de aprendizaje contribuir a su labor,Comentarios contribución a su labor,2.1 Involucrado previamente en comunidades de aprendizaje,2.2 Comunidad de aprendizaje previa,3.01 Sé cómo resolver los problemas técnicos cuando fallan las TIC,3.02 Puedo aprender sobre nuevas tecnologías fácilmente,3.03 Sé cómo usar las TIC con los estudiantes en clase,4. Comentarios primera sección,...,1.5 Puedo evidenciar que hombres y mujeres poseemos capacidades intelectuales semejantes,1.6 Puedo encontrar apoyo de otros y otras docentes para el diseño de mis cursos,1.7 Puedo recibir retroalimentación de mis prácticas pedagógicas,1.8 Aprendo a evaluar mis vacíos y fortalezas conceptuales,1.9 Puedo evidenciar habilidades y destrezas de las mujeres en áreas stem,1.10 Puedo hablar con otros y otras docentes sobre el diseño de cursos,1.11 Aprendo sobre nuevos temas en comunidad,1.12 Puedo conocer diversas plataformas de comunicación para establecer redes de trabajo,1.13 Puedo aprender a ser líder en comunidades de aprendizaje,1.14 Las comunidades de aprendizaje no aportan significativamente a mi labor docente
1,14/06/2022 17:51:03,4.362611e+07,Puedo encontrar apoyo de mis colegas para reso...,NaN,Si,Niños y niñas programación,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,14/06/2022 18:00:12,8.019614e+07,Puedo encontrar apoyo de mis colegas para reso...,La red de aprendizaje comparte experiencias pe...,Si,Comunidad IB bachillerato internacional y la c...,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,...,NO,SI,SI,NO,NO,NO,NO,NO,NO,NO
3,14/06/2022 18:02:47,4.556467e+07,"Aprendo sobre nuevos temas en comunidad, Permi...",NaN,No,NaN,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,De acuerdo,...,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO
4,14/06/2022 19:12:37,6.345713e+07,Puedo encontrar apoyo de mis colegas para reso...,Mejorar cada dia como docentes.,No,NaN,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,NO
5,14/06/2022 19:50:57,4.044457e+07,Puedo encontrar apoyo de mis colegas para reso...,NaN,No,NaN,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Netro,...,NO,SI,NO,NO,SI,NO,NO,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,22/06/2022 16:36:21,5.293052e+07,Puedo encontrar apoyo de mis colegas para reso...,Es importante mantener la motivación y las opo...,No,NaN,De acuerdo,De acuerdo,De acuerdo,Totalmente de acuerdo,...,SI,NO,NO,SI,NO,NO,NO,NO,NO,NO
198,24/06/2022 8:11:14,7.170453e+07,Permiten mejorar mis prácticas pedagógicas por...,Intercambiar conocimientos,No,NaN,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,...,SI,NO,SI,NO,SI,NO,NO,NO,NO,NO
199,26/06/2022 22:19:19,1.035233e+09,Puedo encontrar apoyo de mis colegas para reso...,Considero que las comunidades de aprendizaje s...,No,NaN,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,NO
200,27/06/2022 19:20:01,4.562300e+07,"Aprendo sobre nuevos temas en comunidad, Permi...",NaN,Si,en areas básicas,Totalmente en desacuerdo,Netro,Netro,De acuerdo,...,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO


In [11]:
cap.to_excel("Pretest_CAP.xlsx")